# Amazon SageMaker Ground Truth Demonstration for Ranking Model Responses


1. [Introduction](#Introduction)
2. [Prerequisites](#Prerequisites)
    1. [Create a Work Team](#Create-Workteam)
    2. [IAM Role Set up](#IAM-role-set-up)
    3. [Prepare the data](#Prepare-the-data)
3. [Run a Ground Truth labeling job](#Run-a-Ground-Truth-labeling-job)
    1. [Specify Parameters for Labeling Job](#Specify-Labeling-Parameters)
    2. [Create the instruction template](#Create-the-instruction-template)
    3. [Create a Labeling Job](#Create-a-Labeling-Job)
    4. [Gather Human feedback through labeling portal](#Gather-human-feedback-through-labeling-portal)
    5. [Monitoring Labeling Job Status](#Monitoring-Labeling-Job-Status)
4. [Post Processing and Analysis of SageMaker GroundTruth Labeling Job Results](#Post-Processing-and-Analysis-of-SageMaker-Ground-Truth-Labeling-Job-Results)
    1. [Merging Worker Responses](#Merging-Worker-Responses-into-a-Consolidated-JSON-File)
    2. [Analyzing Time Spent on Labeling Tasks](#Analyzing-Time-Spent-on-Labeling-Tasks)
    3. [Consolidating Worker Responses and Output Manifest](#Consolidating-Worker-Responses-and-Output-Manifest)
    4. [Reordering Model Responses Based on Human Evaluation](#Reordering-Model-Responses-Based-on-Human-Evaluation)

## Introduction

This notebook guides you through the process of setting up a Human-in-the-Loop (HITL) workflow for Reinforcement Learning from Human Feedback (RLHF) using Amazon SageMaker Ground Truth's Text Ranking template. The objective is to rank model-generated responses using human annotators. This labeling UI facilitates human annotators to evaluate and rank model responses based on various dimensions such as "clarity," "bias," and "accuracy." The UI presents prompts alongside responses from multiple models or the same model with different parameters, enabling detailed comparative analysis.

Some key use cases for this labeling UI include:

<b>Model selection:</b> Customers can use the generated rankings to determine which pre-existing model best suits their specific needs and use cases.

<b>Fine-tuning guidance:</b> The dimensional rankings provide detailed feedback that can inform the fine-tuning process of custom language models, helping to improve performance in targeted areas.

<b>Bias detection and mitigation:</b> By including bias as an evaluation dimension, organizations can identify and address potential biases in model outputs.

<b>Competitive analysis:</b> Companies can use this UI to benchmark their models against competitors or industry standards.

<b>User preference analysis:</b> By involving end-users in the annotation process, companies can gain insights into user preferences and expectations.

<b>Domain-specific optimization:</b> The flexibility to define custom dimensions allows for evaluation tailored to specific domains or industries.

<b>Iterative model improvement:</b> Regular use of this UI can facilitate an iterative approach to model development, with each round of feedback informing subsequent improvements.

The steps include  setting up necessary Ground Truth pre-requisites, downloading a JSON with prompts and responses, converting it to a Ground Truth input manifest, creating a worker task template, creating and monitoring a labeling job, and post-processing the results to deliver a consolidated dataset with ranked responses.

## Prerequisites

You will create some of the resources you need to launch a Ground Truth labeling job in this notebook.

Lets get the latest version of SDK, restart kernel and  import some essential libraries to set up the environment for downloading data, handling JSON files, and leveraging AWS services for machine learning workflows.

In [ ]:
!pip install -q --upgrade pip
!pip install awscli -q --upgrade
!pip install botocore -q --upgrade
!pip install sagemaker -q --upgrade
!pip install py7zr
!pip install datasets

# NOTE: Restart Kernel after the above command

In [ ]:
import boto3
import datetime
import json
import sagemaker

### Create-Workteam 
A work team is a group of workers that complete labeling tasks. If you want to preview the worker UI and execute the labeling task you will need to create a private work team, add yourself as a worker to this team, and provide the work team ARN below.

In [ ]:
WORKTEAM_ARN = ""

print(f"This notebook will use the work team ARN: {WORKTEAM_ARN}")

### IAM-role-set-up
The IAM execution role you used to create this notebook instance must have the following permissions:

AWS managed policy AmazonSageMakerGroundTruthExecution. Run the following code-block to see your IAM execution role name. This GIF demonstrates how to add this policy to an IAM role in the IAM console. You can also find instructions in the IAM User Guide: Adding and removing IAM identity permissions.

When you create your role, you specify Amazon S3 permissions. Make sure that your IAM role has access to the S3 bucket that you plan to use in this example. If you do not specify an S3 bucket in this notebook, the default bucket in the AWS region you are running this notebook instance will be used. If you do not require granular permissions, you can attach AmazonS3FullAccess to your role.

In [ ]:
role = sagemaker.get_execution_role()
role_name = role.split("/")[-1]
print("********************************************************************************")
print("The IAM execution role name:", role_name)
print("The IAM execution role ARN:", role)
print("********************************************************************************")
print(
    "IMPORTANT: Make sure this execution role has the AWS Managed policy AmazonGroundTruthExecution attached."
)

### Prepare-the-data
Before we create the labeling job, we need to ensure that the input data is in the format expected by GroundTruth. We use the prompts and responses we collected from our model in the <b>input_prompts.json</b> file to create the manifest file <b>gt_input_manifest_textranking.json</b>. Each row in the manifest file contains an object(prompt-response pair). We upload the file to Amazon S3.

In [ ]:
# Step 1: Load the JSON file
with open('data/input_prompts.json', 'r') as f:
    data = json.load(f)

# Step 2: Convert to the desired format
formatted_data = ""
for item in data:
    entry = {
        "source": item["prompt"],
        "responses": item["responses"]
    }
    formatted_data += json.dumps(entry) + "\n"

# Step 3: Save the output as a new JSON file
with open('gt_input_manifest_textranking.json', 'w') as f:
    f.write(formatted_data)

print("Conversion complete. The file 'gt_input_manifest_textranking.json' is ready.")

#### Upload the Input manifest into S3 
The JSON that is transformed into a Ground Truth input manifest format where each entry contains a "source" field with the prompt and "responses" field with the model responses, is uploaded to an S3 bucket.

In [ ]:
# Initialize SageMaker session and S3 resource
sess = sagemaker.Session()
bucket = sess.default_bucket()
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
region = boto3.session.Session().region_name

# Define the S3 bucket and folder
bucket_name = bucket
prefix = 'genai'
output_file = 'gt_input_manifest_textranking.json'
s3_path = f"{prefix}/{output_file}"

# Upload the file to S3
s3.Bucket(bucket_name).upload_file(output_file, s3_path)
input_manifest_uri = f"s3://{bucket_name}/{s3_path}"

print(f"File uploaded to s3://{bucket_name}/{s3_path}")

## Run-a-Ground-Truth-labeling-job

###  Create-the-instruction-template

The instruction template dictates what is be displayed on the UI at the time when the human raters review the prompt and model responses and provide feedback. It contains instructions to help them perform their task accurately.

The template below advises human raters to review the Prompt and Model responses and rank the latter based on two dimensions specified under "ordinal-ranking-dimensions" - Accuracy and Clarity.

In [ ]:
from IPython.display import display, HTML

def make_template(save_fname="instructions_tr.template"):
    template = """
    <html>
      <head>
        <meta charset="UTF-8" />
        <link rel="stylesheet" href="https://assets.crowd.aws/css/crowd-html-elements-v2.css" />
        <link rel="icon" href="data:image/svg+xml,<svg xmlns=%22http://www.w3.org/2000/svg%22 viewBox=%220 0 100 100%22><text y=%22.9em%22 font-size=%2290%22>&#129351;</text></svg>" />
        <title>Text Ranking Tool</title>
        <script src="https://assets.crowd.aws/crowd-html-elements-v2s.js"></script>
      </head>

      <body>
        <crowd-form id="crowd-form-submit">
          <crowd-text-ranking
            name="textRanking"
            ordinal-ranking-dimensions='[{"name":"Accuracy","allowTie":true},{"name":"Clarity","allowTie":true}]'
            text='{{ task.input.source }}'
            responses='{{ task.input.responses | to_json }}' >
            <short-instructions>
               <p>Rank the following responses from a language model according to the Dimensions on the right panel.</p>
            </short-instructions>
          </crowd-text-ranking>
        </crowd-form>
        <script src="https://assets.crowd.aws/crowd-html-elements.js"></script>
      </body>
    </html>
    """
    with open(save_fname, "w") as f:
        f.write(template)

# Create the template file locally
make_template(save_fname="./instructions_tr.template")

# Define the S3 path
file_name = 'instructions_tr.template'
ui_s3_path = f"{prefix}/{file_name}"
UITEMPLATES3URI = f"s3://{bucket}/{ui_s3_path}"

# Upload the file to S3 using the s3 client
s3_client.upload_file("./instructions_tr.template", bucket, ui_s3_path)

print(f"File uploaded to {UITEMPLATES3URI}")

### Specify-Labeling-Parameters
Specify a Labeling Job Name, Parameters for the Labeling such as TaskTitle, TaskDescription, TaskKeywords and use the CreateLabelingJob API to launch the Ground truth Labeling Job.

In [ ]:
now = datetime.datetime.now()
timestamp_str = now.strftime("%Y%m%d-%H%M%S")
labeling_job_name = "immersionday-genai-text-ranking-" + timestamp_str

### Create-a-Labeling-Job

In [ ]:
client = boto3.client('sagemaker')

client.create_labeling_job(
    LabelingJobName=labeling_job_name,
    LabelAttributeName='label',
    InputConfig={
        'DataSource': {
            'S3DataSource': {
                'ManifestS3Uri': input_manifest_uri #Enter S3 URI of Input Data Json
            }
        }
    },
    OutputConfig={
        'S3OutputPath': f's3://{bucket}/output/' #Enter S3 URI of Output folder
    },
    RoleArn=role, #Enter IAM Sagemaker Execution Role here,
    HumanTaskConfig={
        'WorkteamArn': WORKTEAM_ARN, #Enter Workteam ARN
        'UiConfig': {
            'UiTemplateS3Uri': UITEMPLATES3URI #Enter S3 URI of UI template
        },
        'TaskKeywords': [
            'QnA',
        ],
        'TaskTitle': 'Generative AI - Text Ranking',
        'TaskDescription': "Rank the responses provided by the Models based on the Dimensions",
        'NumberOfHumanWorkersPerDataObject': 1,
        'TaskTimeLimitInSeconds': 60*30,
        'TaskAvailabilityLifetimeInSeconds': 60*60*24*10,
        'MaxConcurrentTaskCount': 100
    })

### Gather-human-feedback-through-labeling-portal

In [ ]:
workforce = client.describe_workforce(WorkforceName="default")
worker_portal_url = 'https://' + workforce["Workforce"]["SubDomain"]


# Display the URL and instructions
display(HTML(f"""
<body>
<h4>Gather human preference data</h4>
<p>Please complete the human evaluation tasks available in the labeling portal.</p>
<p><a href="{worker_portal_url}">{worker_portal_url}</a>
<p><b>Ensure all tasks are completed before proceeding to the next steps in this notebook.<b></p>
<body>
"""))

### Monitoring-Labeling-Job-Status
We track the status of the ongoing labeling job. It is essential to monitor the job's progress and wait for its completion by the annotators. Once the labeling job is finished, we can then proceed to gather feedback from the annotators. This process ensures that we only collect feedback after the entire job is completed, thereby maintaining the accuracy and reliability of the feedback collected.

In [ ]:
client.describe_labeling_job(LabelingJobName=labeling_job_name)

## Post-Processing-and-Analysis-of-SageMaker-Ground-Truth-Labeling-Job-Results
<b>[OPTIONAL]</b>

### Merging-Worker-Responses-into-a-Consolidated-JSON-File

This section focuses on combining all individual worker responses into a single, comprehensive JSON file. We gather the individual JSON files from the "worker-response" folder, each representing a worker's answers to labeling tasks, and merge them into one consolidated file. The resulting consolidated file makes it easier to analyze worker performance, compare responses across different tasks, and prepare the data for further processing or quality checks.

In the output file, each set of responses is accompanied by an index (e.g., [2,1,3]) that represents how human annotators reordered the original responses. For instance, [2,1,3] means the second response was ranked first, the first response second, and the third response remained in its original position.

In [ ]:
import boto3
import json

s3_client = boto3.client('s3')

def merge_json_files(bucket, prefix, output_bucket, output_key):
    merged_data = []

    # List iteration buckets
    iteration_response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix, Delimiter='/')
    for iteration in iteration_response.get('CommonPrefixes', []):
        # List JSON objects in each iteration bucket
        json_response = s3_client.list_objects_v2(Bucket=bucket, Prefix=iteration['Prefix'])
        for obj in json_response.get('Contents', []):
            # Read and append each JSON file, adding the S3 URI or file name
            key = obj['Key']
            json_obj = s3_client.get_object(Bucket=bucket, Key=key)
            json_data = json.loads(json_obj['Body'].read().decode('utf-8'))
            json_data['workerjsonUri'] = f"s3://{bucket}/{key}"  # Add the source S3 URI to the JSON object
            merged_data.append(json_data)  # Append the modified JSON object

    # Write merged data to a new JSON file in S3
    merged_json = json.dumps(merged_data)
    s3_client.put_object(Body=merged_json, Bucket=output_bucket, Key=output_key)
    print(f"Merged JSON file created at https://s3.console.aws.amazon.com/s3/object/{output_bucket}?region={region}&prefix={output_key}")

# Replace with your actual bucket names and prefixes
source_bucket_name = bucket
iteration_prefix = f'output/{labeling_job_name}/annotations/worker-response/iteration-1/'
output_bucket_name = bucket
output_key = f'output/{labeling_job_name}/merged-worker-data.json'

merge_json_files(source_bucket_name, iteration_prefix, output_bucket_name, output_key)

### Analyzing-Time-Spent-on-Labeling-Tasks
This section analyzes the time spent on labeling tasks in our SageMaker Ground Truth job, from the consolidated worker json file. It calculates the total time invested across all tasks, counts the number of completed tasks, and determines the average time per task. These insights help us understand the overall effort of our labeling project, assess its scale, and plan future workflows more effectively. By examining these time metrics, we can make informed decisions about resource allocation and set realistic expectations for upcoming labeling jobs.

In [ ]:
from colorama import Fore, Back, Style, init

init(autoreset=True)  # Initialize colorama

def calculate_and_display_average_time(bucket, key):
    json_obj = s3_client.get_object(Bucket=bucket, Key=key)
    merged_json_data = json.loads(json_obj['Body'].read().decode('utf-8'))
    total_time = 0
    count = 0
    for json_data in merged_json_data:
        if 'answers' in json_data:
            for answer in json_data['answers']:
                if 'timeSpentInSeconds' in answer:
                    total_time += answer['timeSpentInSeconds']
                    count += 1
    
    average_time = total_time / count if count > 0 else 0
    
    # Create a fancy display
    print("\n" + "="*50)
    print(Fore.CYAN + Style.BRIGHT + "     LABELING TASK SUMMARY     ")
    print("="*50 + "\n")
    
    print(Fore.YELLOW + "📊 " + Style.BRIGHT + "Count of Tasks:")
    print(Fore.WHITE + f"   {count:,} tasks")
    
    print(Fore.YELLOW + "\n⏱️ " + Style.BRIGHT + "Total Time Spent:")
    print(Fore.WHITE + f"   {total_time:.2f} seconds")
    print(Fore.WHITE + f"   ({total_time / 3600:.2f} hours)")
    
    print(Fore.YELLOW + "\n⌛ " + Style.BRIGHT + "Average Time per Task:")
    print(Fore.WHITE + f"   {average_time:.2f} seconds")
    
    print("\n" + "="*50)

calculate_and_display_average_time(output_bucket_name, output_key)

### Consolidating-Worker-Responses-and-Output-Manifest

This section consolidates our labeling project data into a comprehensive single file. We merge information from two key sources: the worker responses and the output manifest. By combining the "source" data from the output manifest with the "answers" provided by workers, we create a unified view of all labeling tasks. This consolidation streamlines our data analysis process, allowing us to easily connect each task's original source with its corresponding worker annotations.

In [ ]:
def merge_data_with_manifest(output_manifest_bucket, output_manifest_key, merged_data_bucket, merged_data_key):
    try:
        # Fetch and load the output manifest file
        manifest_obj = s3_client.get_object(Bucket=output_manifest_bucket, Key=output_manifest_key)
        manifest_data = manifest_obj['Body'].read().decode('utf-8').splitlines()
        
        # Load the merged JSON data
        merged_data_obj = s3_client.get_object(Bucket=merged_data_bucket, Key=merged_data_key)
        merged_data = json.loads(merged_data_obj['Body'].read().decode('utf-8'))
        
        # Convert merged data to a dictionary for easier lookup based on workerjsonUri
        merged_data_dict = {item['workerjsonUri']: item for item in merged_data}
        
        final_data = []
        
        for line in manifest_data:
            manifest_entry = json.loads(line)
            worker_response_ref = manifest_entry.get("label-metadata", {}).get("worker-response-ref")
            
            # Find the matching entry in the merged data
            if worker_response_ref in merged_data_dict:
                # Merge the manifest data with the corresponding worker response data
                combined_data = {**manifest_entry, **merged_data_dict[worker_response_ref]}
                final_data.append(combined_data)
        
        # Optionally, write the final merged data to a new file in S3 or handle it as needed
        final_json = json.dumps(final_data)
        final_output_key = f'output/{labeling_job_name}/final-merged-data.json'  # Customize this as needed
        s3_client.put_object(Body=final_json, Bucket=output_manifest_bucket, Key=final_output_key)
        print(f"Final merged JSON file created at https://s3.console.aws.amazon.com/s3/object/{output_manifest_bucket}?region={region}&prefix={final_output_key}")
    
    except s3_client.exceptions.NoSuchKey as e:
        print(f"Error: {e}")

# Replace with your actual bucket names and keys
output_manifest_bucket = bucket
output_manifest_key = f'output/{labeling_job_name}/manifests/output/output.manifest'
merged_data_bucket = bucket
merged_data_key = f'output/{labeling_job_name}/merged-worker-data.json'

merge_data_with_manifest(output_manifest_bucket, output_manifest_key, merged_data_bucket, merged_data_key)

### Reordering-Model-Responses-Based-on-Human-Evaluation
In the output file, each set of responses is accompanied by an index (e.g., [2,1,3]) that represents how human annotators reordered the original responses. The code here matches these index values with the actual responses from our consolidated file. It then reorganizes and displays the full text of these responses in the order determined by the human evaluators. This process allows us to see how human judgment has altered the ranking of model outputs, providing insights into which responses were deemed more appropriate or higher quality by our annotators.

In [ ]:
# Define the S3 bucket and key for the input JSON
bucket_name = bucket
input_key = f'output/{labeling_job_name}/final-merged-data.json'
output_key = f'output/{labeling_job_name}/reordered_responses.json'

# Function to replace rankings with response text, handling tie rankings
def replace_rankings_with_responses(data):
    for item in data:
        responses = item["responses"]
        for answer in item["answers"]:
            for dimension in answer["answerContent"]["ordinalRankingDimensions"]:
                rankings = dimension["responseRankings"]
                # Store the original rankings in a new field
                dimension["responseIndex"] = rankings
                # Create a list to store the response texts based on rankings
                ranked_responses = []
                for rank in sorted(set(rankings)):
                    # Add all responses that match the current rank
                    ranked_responses.extend([responses[i - 1] for i, r in enumerate(rankings, start=1) if r == rank])
                dimension["responseRankings"] = ranked_responses
    return data

# Load the JSON data from S3
def load_json_from_s3(bucket, key):
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    data = json.loads(obj['Body'].read().decode('utf-8'))
    return data

# Save the JSON data to S3
def save_json_to_s3(data, bucket, key):
    json_data = json.dumps(data, indent=2)
    s3_client.put_object(Body=json_data, Bucket=bucket, Key=key)

# Main processing
def main():
    # Load data from S3
    data = load_json_from_s3(bucket_name, input_key)
    
    # Replace rankings with responses
    updated_data = replace_rankings_with_responses(data)
    
    # Save updated data to S3
    save_json_to_s3(updated_data, bucket_name, output_key)
    
    print(f"Updated JSON data has been saved to https://s3.console.aws.amazon.com/s3/object/{bucket_name}?region={region}&prefix={output_key}")

# Execute the main function
if __name__ == "__main__":
    main()
